In [7]:
from sqlalchemy import create_engine, Column, Integer, String, func 
from sqlalchemy.orm import sessionmaker, declarative_base

In [12]:
# Create Engine 
engine = create_engine('sqlite:///students.db', echo=True)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [13]:
# Define Student Model
class Student(Base):
    __tablename__='students'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    marks = Column(Integer)
    address = Column(String)


    def __repr__(self):
        return f"<Student(name={self.name}, marks={self.marks}, city={self.address})>"

In [14]:
# Create the table in the database 

Base.metadata.create_all(engine)

2025-11-30 22:56:14,593 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-30 22:56:14,594 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2025-11-30 22:56:14,595 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-30 22:56:14,597 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("students")
2025-11-30 22:56:14,598 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-30 22:56:14,599 INFO sqlalchemy.engine.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	marks INTEGER, 
	address VARCHAR, 
	PRIMARY KEY (id)
)


2025-11-30 22:56:14,600 INFO sqlalchemy.engine.Engine [no key 0.00076s] ()
2025-11-30 22:56:14,608 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
# Insert Data
session.query(Student).delete()

new_students = [
    Student(name="Kamal", marks=85, address="Kolkata"),
    Student(name="Riya", marks=92, address="Delhi"),
    Student(name="Arjun", marks=78, address="Mumbai"),
    Student(name="Sneha", marks=88, address="Kolkata"),
    Student(name="Rahul", marks=65, address="Bangalore")
]

session.add_all(new_students)
session.commit()
print("✅ 5 Students inserted successfully.\n")

2025-11-30 22:56:37,235 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-30 22:56:37,237 INFO sqlalchemy.engine.Engine DELETE FROM students
2025-11-30 22:56:37,237 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
2025-11-30 22:56:37,242 INFO sqlalchemy.engine.Engine INSERT INTO students (name, marks, address) VALUES (?, ?, ?) RETURNING id
2025-11-30 22:56:37,242 INFO sqlalchemy.engine.Engine [generated in 0.00012s (insertmanyvalues) 1/5 (ordered; batch not supported)] ('Kamal', 85, 'Kolkata')
2025-11-30 22:56:37,243 INFO sqlalchemy.engine.Engine INSERT INTO students (name, marks, address) VALUES (?, ?, ?) RETURNING id
2025-11-30 22:56:37,243 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/5 (ordered; batch not supported)] ('Riya', 92, 'Delhi')
2025-11-30 22:56:37,245 INFO sqlalchemy.engine.Engine INSERT INTO students (name, marks, address) VALUES (?, ?, ?) RETURNING id
2025-11-30 22:56:37,246 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/5 (ordered; batch not suppor

In [16]:
# Query Data
top_scorer = session.query(Student).order_by(Student.marks.desc()).first()
print(f"\n1st in the class is : {top_scorer.name} got marks {top_scorer.marks} in {top_scorer.address}")

# Students From a city
city_students = session.query(Student).filter(Student.address == "Kolkata").all()
print("\nStudents from Kolkata:")
for s in city_students:
    print(f"✅ {s.name} got marks {s.marks}")

# Average Marks
avg_marks = session.query(func.avg(Student.marks)).scalar()
print(f"\nAverage marks in the class: {avg_marks:.2f}")

# Students with marks greater than 80

good_students = session.query(Student).filter(Student.marks > 80).all()
print("\nStudents with marks greater than 80:")
for s in good_students:
    print(f"✅ {s.name} got marks {s.marks}")


# Update the data in the database through the ORM
student_to_update = session.query(Student).filter(Student.name == "Kamal").first()
if student_to_update:
    student_to_update.marks = 90
    session.commit()
    print(f"\n✅ Updated Kamal's marks to {student_to_update.marks}\n")

# Delete the data in the database through the ORM
session.query(Student).filter(Student.name == "Rahul").delete()
session.commit()
print("✅ Deleted Rahul's data\n")

# Aggregation (Task: Count the total students)
total_students = session.query(func.count(Student.id)).scalar()
print(f"Total number of students: {total_students}\n")



2025-11-30 22:57:32,544 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-30 22:57:32,548 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.marks AS students_marks, students.address AS students_address 
FROM students ORDER BY students.marks DESC
 LIMIT ? OFFSET ?
2025-11-30 22:57:32,550 INFO sqlalchemy.engine.Engine [generated in 0.00169s] (1, 0)

1st in the class is : Riya got marks 92 in Delhi
2025-11-30 22:57:32,555 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.marks AS students_marks, students.address AS students_address 
FROM students 
WHERE students.address = ?
2025-11-30 22:57:32,556 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ('Kolkata',)

Students from Kolkata:
✅ Kamal got marks 85
✅ Sneha got marks 88
2025-11-30 22:57:32,561 INFO sqlalchemy.engine.Engine SELECT avg(students.marks) AS avg_1 
FROM students
2025-11-30 22:57:32,562 INFO sqlalchemy.eng